# Tutorial: Single chromosome optimization using OpenMiChroM


## This tutorial performs optimization in the MiChroM Parameters (Second-order optimization -> Hessian inversion)

*The first step is to import the OpenMiChroM modules.*

To install OpenMM and OpenMiChroM, follow the [instalation guide](https://open-michrom.readthedocs.io/en/latest/#).

The inputs and apps used in this tutorial can be downloaded [here](https://github.com/junioreif/OpenMiChroM/tree/main/Tutorials/MiChroM_Optimization).

<font color='red'>Types optimization is available starting in the OpenMichroM version 1.0.7.</font> 

In [ ]:
# OpenMiChroM simulation module
from OpenMiChroM.ChromDynamics import MiChroM
# Optimization of MiChroM parameters module
from OpenMiChroM.Optimization import CustomMiChroMTraining
# Analysis tools module
from OpenMiChroM.CndbTools import cndbTools

# Extra modules to load and plot .dense file 
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd
import h5py
from scipy.optimize import curve_fit
import os

*The second step is to have a look on the experimental Hi-C*

A Hi-C file is required for the analysis and training of the MiChroM Potentials (Types and Ideal Chromosome). The file format chosen here is a matrix .txt file (we call it the dense file).

For this tutorial, we will use chromosome 10 from GM12878 in 100 kb resolution. To extract it from the .hic file, we can use juicer_tools with the folowing command. Please note that the juicer_tools java file is located at the `apps` folder located at the `$HOME` directory.

In [ ]:
#%%bash
#java -jar ~/apps/juicer_tools_1.22.01.jar dump observed Balanced -d https://hicfiles.s3.amazonaws.com/hiseq/gm12878/in-situ/combined.hic 10 10 BP 100000 input/chr10_100k.dense
# download the file in github

This command downloads the .hic from the web and extracts the chromosome 10 in .dense format to the folder "input".

You can get more information about it at the [JuicerTools](https://github.com/aidenlab/juicer) documentation.

Visualize the .dense file.

In [ ]:
filename = 'input/chr10_100k.dense'
hic_file = np.loadtxt(filename)

r=np.triu(hic_file, k=1) 
r[np.isnan(r)]= 0.0
r = normalize(r, axis=1, norm='max') 
rd = np.transpose(r) 
r=r+rd + np.diag(np.ones(len(r)))
print("number of beads: ", len(r))
plt.matshow(r,norm=mpl.colors.LogNorm(vmin=0.0001, vmax=r.max()),cmap="Reds")  
plt.colorbar()

This Hi-C map has a resolution of 100 kb per bead, so the chromosome 10 model has a polymer chain with a total of 1356 beads.

The next step is to extract the sequence file containing the A/B compartments along the sequence by using the eigenvector decomposition. <br>

Using the juicertools, you can extract the eigenvector file. The eigenvector has values both negatives and positives and here we will arbitrary set positives as B and negatives as A. <br>

For more details about how it works, please take a look on this paper: https://pubs.acs.org/doi/full/10.1021/acs.jpcb.1c04174

In [ ]:
#%%bash
#java -jar ~/apps/juicer_tools_1.22.01.jar eigenvector -p Balanced https://hicfiles.s3.amazonaws.com/hiseq/gm12878/in-situ/combined.hic 10 BP 100000 input/chr10_100k.eigen
# download the file in github

And visualize the .eigen file.

In [ ]:
eigen = np.loadtxt("input/chr10_100k.eigen")

index = np.arange(eigen.size)
df = pd.Series(eigen, index=index)

A1_df = pd.Series(np.zeros(eigen.size), index=index) 
A1_df[df < 0] = df[df < 0]
B1_df = pd.Series(np.zeros(eigen.size), index=index) 
B1_df[df > 0] = df[df > 0]

NA_df = df.isna()
# y = sc.signal.savgol_filter(A1,20, 2)

fig, axes = plt.subplots(ncols=1, nrows=2,
                         figsize=(6.4*3/2, 4.8*2/3),
                         gridspec_kw={'height_ratios': [1, 10]})
fig.tight_layout()

fig.subplots_adjust(hspace=0.05) # top=0.92, bottom=0.01, wspace=0.)

# sequence
A11_df = A1_df.copy()
A11_df[A1_df<0] = 1
B11_df = B1_df.copy()
B11_df[B1_df>0] = 1

axes[0].axis('off')
width=1
axes[0].bar(A11_df.index, A11_df,
       width,
       color="tab:red")
axes[0].bar(B11_df.index, B11_df,
       width,
       color="tab:blue")

axes[0].set_xlim([-50,eigen.size + 50])

# Eigen PC1
step=1
axes[1].fill_between(A1_df.index[::step], A1_df[::step],  0,
                facecolor="tab:red",
                alpha=0.8,
                label="A")
axes[1].fill_between(B1_df.index[::step], B1_df[::step],  0,
                facecolor="tab:blue",
                alpha=0.8,
                label="B")

axes[1].set_ylim([-0.05,0.05])
axes[1].set_xlim([-50,eigen.size + 50])

axes[1].set_ylabel('Eigen values, PC1')
axes[1].set_xlabel('Genomic distance (50 kb)')

axes[1].legend(title='compartments', bbox_to_anchor=(1.,1.45), 
               frameon=False)

print("Number of beads: A1 = ", A1_df[A1_df<0].shape[0], 
      ", B1 = ", B1_df[B1_df>0].shape[0])

From the .eigen, file we can create the A/B sequence file.

In [ ]:
import pandas as pd

# Caminho para o arquivo .eigen
file_path = "input/chr10_100k.eigen"

# Lê o arquivo .eigen
df = pd.read_csv(file_path, sep="\s+", header=None, names=["value"])

# Classifica os valores em "A1" (se menor que 0) e "B1" (se maior ou igual a 0)
df["classification"] = df["value"].apply(lambda x: "A1" if x < 0 else "B1")

# Adiciona números sequenciais (1, 2, 3, ...)
df.reset_index(inplace=True)
df.rename(columns={"index": "line_number"}, inplace=True)
df["line_number"] += 1  # Ajusta para começar do 1

# Salva o resultado no arquivo de saída
output_path = "input/seq_chr10_100k.txt"
df[["line_number", "classification"]].to_csv(output_path, index=False, header=False, sep="\t")

# Conta as ocorrências de "A1" e "B1" e exibe no terminal
classification_counts = df["classification"].value_counts()
print(f"Arquivo gerado: {output_path}")
print("Número de classificações:")
print(classification_counts)

## 1\# Types Interaction Opmization

In addition to the homopolimer term, MiChroM energy function has two main terms: the type-to-type and Ideal Chromosome interactions. 

<!--![](https://pubs.acs.org/cms/10.1021/acs.jpcb.1c04174/asset/images/medium/jp1c04174_m010.gif)-->

${ U }_{\text{MiChroM} }(\vec { r } )={ U }_{ HP }\left( \vec { r }  \right) +\sum _{\tiny{ \begin{matrix}  k\ge l \\ k,l \in  \text{Types} \end{matrix} } }{ { \alpha }_{ kl } } \sum _{\tiny{ \begin{matrix} i \in \{ \text{Loci of Type k}\}  \\ j \in  \{ \text{Loci of Type l} \}  \end{matrix} }} f({ r }_{ ij }) +  \sum _{ d=3 }^{ { d }_{ cutoff } }{ \gamma (d)\sum _{ i }{ f({ r }_{ i,i+d }) }  }$

In this section, the types interaction terms will be minimized.

The pipeline to perform the Types Potential Optimization is the following:

1. Run a long simulation using the homopolymer + customTypes potential of the `OpenMiChroM` module. 
    The first iteration can start with all parameters equal to zero or set to an initial guess.

2. Get the frames from this simualtion to perform the inversion for Types.

3. In the end of the inversion, new values to types interactions will be produced.

4. Calcule the error/tolerance between the simulated and experimental parameters. If it is above the treshold, re-do steps 1-3 until reaching the treshold (usually 10% or 15% is enougth).

The Types file is a `.txt` with a matrix labeled with the values for each interaction. In this tutorial, we are training A and B types. <br> 

Lets create the initial file with this format:
<pre><code>
A,B
0,0
0,0
</pre></code>
For this matrix, we have AA AB BA BB interactions.

Save it as `lambda_0`

In [ ]:
# Criar o conteúdo do arquivo
conteudo = """A1,B1
0,0
0,0"""

# Nome da pasta e do arquivo
nome_pasta = "input"
nome_arquivo = "lambda_0.txt"

# Verificar se a pasta existe, se não, criar
if not os.path.exists(nome_pasta):
    os.makedirs(nome_pasta)

# Caminho completo para o arquivo
caminho_arquivo = os.path.join(nome_pasta, nome_arquivo)

# Escrever o conteúdo no arquivo
with open(caminho_arquivo, "w") as arquivo:
    arquivo.write(conteudo)

print(f"O arquivo {nome_arquivo} foi salvo na pasta '{nome_pasta}' com sucesso!")

With all the required inputs, lets perform a simulation for iteration 0.

In MiChroM initiation there are some variables to setup:

**time_step=0.01** (the time step using for integration, default is 0.01)<br>
**temperature=1** (Set the temperature of your simulation)<br>
**name='opt_chr10_100K'** (the simulation name)<br>

In [ ]:
simulation = MiChroM(name='opt_chr10_100K', temperature=1.0, timeStep=0.01)

Now you need to setup the platform that you will use, the options are:

**platform="OpenCL"** (it can also be CUDA or CPU depending of your system)<br>
**GPU="0" (optional)** (if you have more than one GPU device, you can set the GPUs that you want ["0", "1",...,"n"])<br>

In [ ]:
simulation.setup(platform="CUDA")

Set the folder name where the output will be saved.

In [ ]:
simulation.saveFolder('iteration_0')

The next step is to setup your chromosome sequence and initial configuration.

In [ ]:
mychro = simulation.createRandomWalk(ChromSeq="input/seq_chr10_100k.txt")

Load the initial structure in the `simulation` object.

In [ ]:
simulation.loadStructure(mychro, center=True)

Now it is time to include the force field in the `simulation` object

Lets separate forces in two sets:

**Homopolymer Potentials**  

In [ ]:
simulation.addFENEBonds(kFb=30.0)
simulation.addAngles(kA=2.0)
simulation.addRepulsiveSoftCore(eCut=4.0)

**Chromosome Potentials**

In this tutorial, it is used the CustomTypes potential. <br>
Here we need to pass a file that contains a matrix of interactions for each other different type of chromosome.
To check that, you can look on the [OpenMiChroM](https://open-michrom.readthedocs.io/en/latest/OpenMiChroM.html#OpenMiChroM.ChromDynamics.MiChroM.addCustomTypes) documentation. <br>

In [ ]:
simulation.addCustomTypes(mu=3.22, rc = 1.78, TypesTable='input/lambda_0.txt')

Note: these valeus for $\mu$ and $r_c$ were calculated for human GM12878 cells and can be changed for other species.

The last potential to be added is the spherical restraint in order to collapse the initial structure.

In [ ]:
simulation.addFlatBottomHarmonic(kR=5*10**-3, nRad=8.0)

Now we will run a short simulation in order to get a collapsed structure.

There are two variables that control the chromosomes simulation steps:

**block:** The number of steps performed in each cycle.<br>
**n_blocks:** The number of blocks that will be perfomed. <br>

In this example, to perfom the collapsing we will run $1\times10^3 \times  10^2 = 1\times10^5$ steps

In [ ]:
prodution = 10**6

We can save the radius of gyration of each block to observe the convergence into the collapsed state (the time required here depends on the size of your chromosome).

In [ ]:
simulation.createSimulation()
simulation.createReporters(statistics=True, traj=True, trajFormat="cndb", energyComponents=False, interval=1000)
simulation.createReporters(statistics=True, traj=True, trajFormat="gro", energyComponents=False, interval=1000)
simulation.run(nsteps=prodution, report=False)

Some details about the output for each block performed:

**bl=0** is the number of each block, in this case we set increment=False, so the number of steps is not accounted.<br>
**pos\[1\]=\[X,Y,Z\]** the spatial position for the first bead. <br>
**dr=1.85** show the average positions displacement in each block (in units os sigma). <br>
**t=1000.0 ps** the time step. in this case we set increment=False, so the number of steps is not accounted.<br>
**kin=1.48** is the kinect energy of the system.<br>
**pot=21.60** is the potential energy of the system. <br>
**RG=11.809** is the radius of gyration in the end of each block. <br>
**SPS=1538** is the steps per second of each block. A way to look how fast the computations are being performed. In this case, it was executed in a MB Pro 2021 with a Apple M1 Max.

Check the convergence of the radius of gyration:

In [ ]:
cndbTools = cndbTools()
chr10_traj = cndbTools.load('iteration_0/opt_chr10_100K_0.cndb')

In [ ]:
first_frame = min([int(key) for key in chr10_traj.cndb.keys() if key != 'types'])
last_frame = max([int(key) for key in chr10_traj.cndb.keys() if key != 'types'])
chr10_xyz = cndbTools.xyz(frames=[first_frame, last_frame,1], XYZ=[0,1,2])
chr10_RG = cndbTools.compute_RG(chr10_xyz)
plt.plot(chr10_RG)
plt.ylabel(r'Radius of Gyration ($\sigma$)',fontsize=11)
plt.xlabel(r'Simulation Frames',fontsize=11)

The next step is to remove the restraint force in order to run the sampling simulation. It also adds a confinement potential with density = 0.1 (volume fraction).

In [ ]:
# Remove Flat initialized in Collapse
simulation.removeFlatBottomHarmonic()

# Add a confinement potential with density=0.1 (volume fraction)
simulation.addSphericalConfinementLJ()

Initiate the `optimization` object for this tutorial section.

In [ ]:
optimization = CustomMiChroMTraining(ChromSeq="input/seq_chr10_100k.txt", 
                                     TypesTable='input/lambda_0.txt',
                                     mu=3.22, rc = 1.78)

The next step is to perform a long simulation to feed the optimization parameters. <br>

In order to get a good inversion calcultation, it is important to have around $1\times10^5$ frames from a certain amount of different replicas. For example, $20$ replicas with $5.000$ saved frames from each. <br> 

This can take some time, so in this tutorial we will use just 1 replica of $5.000$ frames saved every $1.000$ steps. <br>

    block = 1000
    n_blocks = 5000

In [ ]:
prodution = 10**7
simulation.run(nsteps=prodution, report=False)

# feed the optimization with the last chromosome configuration
optimization.prob_calculation_types(simulation.getPositions())

In the end of each replica simulation, we need to save some important values required to calculate the inversion. <br>

We are saving these files using the H5 compression because it is faster to write/read. <br>

**Note**: attention to this step. We have these 4 files for each replica for each iteration step. Be organized!

In [ ]:
replica="1"

with h5py.File(simulation.folder + "/Nframes_" + replica + ".h5", 'w') as hf:
    hf.create_dataset("Nframes",  data=optimization.Nframes)

with h5py.File(simulation.folder + "/Pold_" + replica + ".h5", 'w') as hf:
    hf.create_dataset("Pold",  data=optimization.Pold)

# Specific for Types minimization
with h5py.File(simulation.folder + "/Pold_type_" + replica + ".h5", 'w') as hf:
    hf.create_dataset("Pold_type",  data=optimization.Pold_type)

with h5py.File(simulation.folder + "/PiPj_type_" + replica + ".h5", 'w') as hf:
    hf.create_dataset("PiPj_type",  data=optimization.PiPj_type)

The first part of the optimization is finished. Look inside the output folder to see these 4 files that will be used in next step.

In [ ]:
# Defina o nome da pasta
nome_pasta = "iteration_0"

# Verificar se a pasta existe
if os.path.exists(nome_pasta):
    # Listar todos os arquivos na pasta
    arquivos = [arquivo for arquivo in os.listdir(nome_pasta) if arquivo.endswith(".h5")]
    
    # Printar os arquivos encontrados
    if arquivos:
        print("Arquivos .h5 encontrados na pasta:")
        for arquivo in arquivos:
            print(arquivo)
    else:
        print("Nenhum arquivo .h5 encontrado na pasta.")
else:
    print(f"A pasta '{nome_pasta}' não existe.")

The next part is the inversion. It is quite simple, just feed the optmization object with all replicas and make the inversion to get the new lambda file.

In [ ]:
inversion = CustomMiChroMTraining(ChromSeq="input/seq_chr10_100k.txt", 
                                  TypesTable='input/lambda_0.txt',
                                  mu=3.22, rc = 1.78)

In [ ]:
iterations = "iteration_0"
replica    = "1"

with h5py.File(iterations + "/Nframes_" + replica + ".h5", 'r') as hf:
    inversion.Nframes += hf['Nframes'][()]

with h5py.File(iterations + "/Pold_" + replica + ".h5", 'r') as hf:
    inversion.Pold += hf['Pold'][:]

# For Types
with h5py.File(iterations + "/Pold_type_" + replica + ".h5", 'r') as hf:
    inversion.Pold_type += hf['Pold_type'][:]

with h5py.File(iterations + "/PiPj_type_" + replica + ".h5", 'r') as hf:
    inversion.PiPj_type += hf['PiPj_type'][:]

With the parameters of all replicas, we calculate the inversion and get the new lambdas. It is calculated by using the Newton Method: <br>

$\lambda_1 = \lambda_0 - \delta\times\lambda_{actual}$

$\delta$ is the learning rate or damp, it can be adjusted in order to get values between [-1:0]. The average value of MiChroM's parameters for Human GM12878 is -0.3 when generating $1\times10^5$ configurations among different replicas.

In [ ]:
# Passo 4: Executar get_lambdas_IC com norm=True implicitamente
lambdas = inversion.get_lambdas_IC(exp_map="input/chr10_100k.dense", damp=3e-7)

# Passo 5: Exibir resultado
print("Matriz de lambdas final:")
print(lambdas)

In [ ]:
dense_exp = np.loadtxt("chr10_100k.dense")
print(dense_exp)

Save the new lambda (`lambda_1`) and other files to analyze the inversion.

In [ ]:
iteration = "0"

# Probabilities of As/Bs in the simulation and experiment
phi_sim = inversion.calc_phi_sim_types().ravel()
phi_exp = inversion.calc_phi_exp_types().ravel()

np.savetxt('iteration_0/phi_sim_' + iteration, phi_sim)
np.savetxt('iteration_0/phi_exp', phi_exp)

plt.plot(phi_sim[phi_sim>0], 'o', label="simulation")
plt.plot(phi_exp[phi_exp>0], 'o', label="experiment")
plt.ylabel(r'Contact probability, $\phi$')
labels_exp = ["AA", "AB", "BB"]
plt.xticks(np.arange(len(labels_exp)), labels_exp)
plt.legend()

# Save and plot the simulated Hi-C
dense_sim = inversion.get_HiC_sim()
np.savetxt('iteration_0/hic_sim_' + iteration + '.dense', dense_sim)

dense_exp = np.loadtxt("input/chr10_100k.dense")
dense_exp[np.isnan(dense_exp)] = 0.0
dense_exp = normalize(dense_exp, axis=1, norm='max')
r = np.zeros(dense_sim.size).reshape(dense_sim.shape)
r = np.triu(dense_exp, k=1) + np.tril(dense_sim, k=-1) + np.diag(np.ones(len(r)))

plt.matshow(r, 
            norm=mpl.colors.LogNorm(vmin=0.0001, 
                                    vmax=dense_sim.max()), 
            cmap="Reds")

# Save the new lambda file
lambdas.to_csv("iteration_0/lambda_1", index=False)

Redo these steps using the new lambda file (`lambda_1`) as input for Types potential in the next iteration. 

The tolerance can be calculate using `phi_sim_1` and `phi_exp` by the equation:<br>

$tolerance = \frac{|\phi_{sim}-\phi_{exp}|}{\phi_{exp}}$

This is appended in to file  `tolerance_and_pearson_types`.

In [ ]:
%%bash
cat tolerance_and_pearson_types

We included a folder named `scripts` that has some `.py` and `.slurm` files that can be used to run this optimization in parallel using slurm clusters. It is located in our github repository https://github.com/junioreif/OpenMiChroM.

## 2\# Ideal Chromosome Interaction Opmization

Following the previous section protocoal, now it is time to learn how to minimize the Ideal Chromosome (IC) interaction, the last term of our hamiltonian

<!--![](https://pubs.acs.org/cms/10.1021/acs.jpcb.1c04174/asset/images/medium/jp1c04174_m010.gif)-->

${ U }_{\text{MiChroM} }(\vec { r } )={ U }_{ HP }\left( \vec { r }  \right) +\sum _{\tiny{ \begin{matrix}  k\ge l \\ k,l \in  \text{Types} \end{matrix} } }{ { \alpha }_{ kl } } \sum _{\tiny{ \begin{matrix} i \in \{ \text{Loci of Type k}\}  \\ j \in  \{ \text{Loci of Type l} \}  \end{matrix} }} f({ r }_{ ij }) +  \sum _{ d=3 }^{ { d }_{ cutoff } }{ \gamma (d)\sum _{ i }{ f({ r }_{ i,i+d }) }  }$

The pipeline to perform the IC minimization is similar to the previous one:

1. Run a long simulation using the homopolymer + types potential + addCustomIC of the `OpenMiChroM` module. 
    The first iteration can start with all parameters equal to zero or set to an initial guess.

2. Get the frames from this simualtion to perform the inversion.

3. In the end of the inversion, new values to the interactions will be produced.

4. Calcule the tolerance between the simulated and experimental parameters. If it is above the treshold, re-do steps 1-3 until reaching the treshold (usually 10% or 15% is enougth).

The IC file is a single column `.txt` with the values for each interaction. The file lenght should the genome distance (`d`) that you want to train. In the above equation, it goes from `d = 3` to a certain cutoff that normaly would be 200 for the human chromosome. <br> 

Lets create the initial file with this format:
<pre><code>
0
0
.
.
.
0
</pre></code>

Save it as `lambda_0`.

In [ ]:
%%bash
rm input/lambda_0_IC &> /dev/null
for (( i=3; i<=1356; i++ )); do echo 0 >> input/lambda_IC_0; done
wc -l input/lambda_IC_0

Again, create the simulation that will colapse the initial structure.

In [ ]:
simulation_ic = MiChroM(name='opt_ic_chr10_100K', temperature=1.0, time_step=0.01)

simulation_ic.setup(platform="OpenCL")
simulation_ic.saveFolder('iteration_ic_0')
mychro_ic = simulation_ic.createSpringSpiral(ChromSeq="input/seq_chr10_100k.txt")
simulation_ic.loadStructure(mychro_ic, center=True)

# Adding Potentials section

# **Homopolymer Potentials**  
simulation_ic.addFENEBonds(kfb=30.0)
simulation_ic.addAngles(ka=2.0)
simulation_ic.addRepulsiveSoftCore(Ecut=4.0)

# **Chromosome Potentials**
simulation_ic.addTypetoType()
simulation_ic.addCustomIC(IClist="input/lambda_IC_0",
                          dinit=3, dend=200) 

# The restriction term for colapsing the beads
simulation_ic.addFlatBottomHarmonic(kr=5*10**-3, n_rad=8.0)

Execute the simulation to colapse the initial structure

In [ ]:
block    = 1000
n_blocks = 200

rg_ic = []

for _ in range(n_blocks):
    simulation_ic.runSimBlock(block, increment=False)
    rg_ic.append(simulation_ic.chromRG())

#save a collapsed structure in pdb format for inspection
simulation_ic.saveStructure(mode = 'pdb')

plt.plot(rg_ic)

It is time to remove the flat bottom constraint and include the spherical confinement mimicking the nuclear density. 

In [ ]:
# Remove Flat initialized in Collapse
simulation_ic.removeFlatBottomHarmonic()

# Add a confinement potential with density=0.1 (volume fraction)
simulation_ic.addSphericalConfinementLJ()

And create the IC optimization object.

In [ ]:
optimization_ic = CustomMiChroMTraining(ChromSeq="input/seq_chr10_100k.txt", 
                                        TypesTable='input/lambda_0',
                                        IClist="input/lambda_IC_0",
                                        dinit=3, dend=200)

The next step is to perform a long simulation to feed the optimization parameters. <br>

In order to get a good inversion calcultation, it is important to have around $1\times10^5$ frames from a certain amount of different replicas. For example, $20$ replicas with $5.000$ saved frames from each. <br> 

This can take some time, so in this tutorial we will use just 1 replica. <br>

In [ ]:
block    = 1000
n_blocks = 5000

for _ in range(n_blocks):
    # perform 1 block of simulation
    simulation_ic.runSimBlock(block, increment=True)
    
    # feed the optimization with the last chromosome configuration
    optimization_ic.prob_calculation_IC(simulation_ic.getPositions())


It is time to save the important files required for the inversion. <br>

These files are prefered to be saved using the H5 compression. <br>

**Note**: attention to this step. We have these 4 files for each replica for each iteration step. Be organized!

In [ ]:
replica="1"

with h5py.File(simulation_ic.folder + "/Nframes_" + replica + ".h5", 'w') as hf:
    hf.create_dataset("Nframes",  data=optimization_ic.Nframes)

with h5py.File(simulation_ic.folder + "/Pold_" + replica + ".h5", 'w') as hf:
    hf.create_dataset("Pold",  data=optimization_ic.Pold)

# Specific for IC minimization
with h5py.File(simulation_ic.folder + "/PiPj_IC_" + replica + ".h5", 'w') as hf:
    hf.create_dataset("PiPj_IC",  data=optimization_ic.PiPj_IC)

In [ ]:
%%bash
ls iteration_ic_0/*.h5

The next part is the inversion. We need to feed the optmization object with all replicas and make the inversion to get the new lambda file.

In [ ]:
inversion_ic = CustomMiChroMTraining(ChromSeq="input/seq_chr10_100k.txt",
                                     TypesTable='input/lambda_0',
                                     IClist="input/lambda_IC_0",    
                                     dinit=3, dend=200)

In [ ]:
iterations = "iteration_ic_0"
replica    = "1"

with h5py.File(iterations + "/Nframes_" + replica + ".h5", 'r') as hf:
    inversion_ic.Nframes += hf['Nframes'][()]

with h5py.File(iterations + "/Pold_" + replica + ".h5", 'r') as hf:
    inversion_ic.Pold += hf['Pold'][:]

# For IC
with h5py.File(iterations + "/PiPj_IC_" + replica + ".h5", 'r') as hf:
    inversion_ic.PiPj_IC += hf['PiPj_IC'][:]

With the parameters of all replicas, we calculate the inversion and get the new lambdas. It is calculated by using the Newton Method: <br>

$\lambda_1 = \lambda_0 - \delta\times\lambda_{actual}$

$\delta$ is the learning rate or damp, it can be adjusted in order to get values between [-1:0]. The average value of MiChroM's parameters for Human GM12878 is -0.3 when generating $1\times10^5$ configurations among different replicas.

In [ ]:
lambdas_ic = inversion_ic.get_lambdas_IC(exp_map="input/chr10_100k.dense",
                                         damp=5e-4)

lambdas_ic.size, lambdas_ic[:5]

Save the new lambda (`lambda_1_ic`) and other files to analyze the inversion.

In [ ]:
iteration = "0"

# Probabilities of As/Bs in the simulation and experiment
phi_sim = inversion_ic.calc_phi_sim_IC()
phi_exp = inversion_ic.calc_phi_exp_IC()

np.savetxt('iteration_ic_0/phi_sim_' + iteration, phi_sim)
np.savetxt('iteration_ic_0/phi_exp', phi_exp)

plt.plot(phi_sim, label="simulation")
plt.plot(phi_exp, label="experiment")
plt.ylabel(r'Contact probability, $\phi$')
plt.xlabel(r'Genomic distance, $d$')
plt.yscale('log')
plt.xscale('log')
plt.legend()

# Save and plot the simulated Hi-C
dense_sim = inversion_ic.get_HiC_sim()
np.savetxt('iteration_ic_0/hic_sim_' + iteration + '.dense', dense_sim)

dense_exp = np.loadtxt(filename)
dense_exp[np.isnan(dense_exp)] = 0.0
dense_exp = normalize(dense_exp, axis=1, norm='max')
r = np.zeros(dense_sim.size).reshape(dense_sim.shape)
r = np.triu(dense_exp, k=1) + np.tril(dense_sim, k=-1) + np.diag(np.ones(len(r)))

plt.matshow(r, 
            norm=mpl.colors.LogNorm(vmin=0.0001, 
                                    vmax=dense_sim.max()), 
            cmap="Reds")

# Save the new lambda file
np.savetxt("iteration_ic_0/lambda_1", lambdas_ic)

The simulated interaction terms (`lambdas_1` data) can be used to fit a model in order to run production simulations with longer cutoffs in the IC energy term. The resulting model is used in the `addCustomIC` method of the `OpenMiChroM` class.

In [ ]:
def objective1(x, a, b, c):
	return -a / np.log(x) - b / x**2 - c / x

y = lambdas_ic
x = np.arange(3, lambdas_ic.size + 3)
plt.scatter(x, y, label='simulation')

# curve fit
popt, pcov = curve_fit(objective1, x, y)
a, b, c = popt
standard_dev = np.sqrt(np.diag(pcov))

# IC generated model 
print(popt, standard_dev)

# plot the fit
x_fit = x
y_fit = objective1(x_fit, a, b, c)

plt.plot(x_fit, y_fit, label='fitting', 
	 	 color='#ff7f0e')

Nice job! You have completed the optimization tutorial of the MiChroM energy terms. 